In [1]:
import snowflake.connector

conn = snowflake.connector.connect(
    user='yangqihuang',
    password='Intelligent106$',
    account='dorcpgt-lbb34148'
    )

In [2]:
cs = conn.cursor()

In [116]:
#Q1
import os
import glob

#combine all 41 files into one file
input = '/home/jovyan/sql_etl_project/Monthly PO Data'
output = '/home/jovyan/sql_etl_project/combined_purchase_data.csv'

csv_files = glob.glob(os.path.join(input, '*.csv'))

with open(output, 'w') as outfile:
    for i, file in enumerate(csv_files):
        with open(file, 'r') as infile:
            if i == 0:
                outfile.write(infile.read())
            else:
                infile.readline()
                outfile.write(infile.read())


In [ ]:
#create project database
cs.execute("CREATE DATABASE IF NOT EXISTS projectdb")
#create project database's schema
cs.execute("CREATE SCHEMA IF NOT EXISTS projectschema")

In [4]:
#use database and schema
cs.execute("USE DATABASE projectdb")
cs.execute("USE SCHEMA projectschema")

In [117]:
#create specific stage for purchase data
cs.execute("CREATE STAGE poh_internal_stage")

In [118]:
#load the combined csv file into the specific stage
cs.execute("PUT file:///home/jovyan/sql_etl_project/combined_purchase_data.csv @poh_internal_stage")

In [119]:
#create table for csv file to copy into
sql11 = '''
CREATE OR REPLACE TABLE poh_raw (
purchaseorderid VARCHAR, 
supplierid VARCHAR, 
orderdate VARCHAR, 
deliverymethodid VARCHAR, 
contactpersonid VARCHAR, 
expecteddeliverydate VARCHAR, 
supplierreference VARCHAR, 
isorderfinalized VARCHAR, 
comments VARCHAR, 
internalcomments VARCHAR, 
lasteditedby VARCHAR, 
lasteditedwhen VARCHAR, 
purchaseorderlineid VARCHAR, 
stockitemid VARCHAR, 
orderedouters VARCHAR, 
description VARCHAR, 
receivedouters VARCHAR, 
packagetypeid VARCHAR, 
expectedunitpriceperouter VARCHAR, 
lastreceiptdate VARCHAR, 
isorderlinefinalized VARCHAR, 
right_lasteditedby VARCHAR, 
right_lasteditedwhen VARCHAR
)
'''
cs.execute(sql11)

In [120]:
#copy into and set format
cs.execute('''CREATE OR REPLACE FILE FORMAT csv_format TYPE = 'CSV' FIELD_OPTIONALLY_ENCLOSED_BY = '"' FIELD_DELIMITER = ',' PARSE_HEADER = FALSE, SKIP_HEADER = 1''')
cs.execute('''COPY INTO poh_raw from @poh_internal_stage FILE_FORMAT = csv_format''')

In [121]:
#delete null columns
cs.execute("ALTER TABLE poh_raw DROP COLUMN comments, internalcomments")

In [122]:
#convert each column's datatype
sql12 = '''
ALTER TABLE poh_raw ADD COLUMN 
purchaseorderid_int INTEGER, 
supplierid_int INTEGER, 
orderdate_date DATE, 
deliverymethodid_int INTEGER, 
contactpersonid_int INTEGER, 
expecteddeliverydate_date DATE, 
isorderfinalized_bol BOOLEAN, 
lasteditedby_int INTEGER, 
purchaseorderlineid_int INTEGER, 
stockitemid_int INTEGER, 
orderedouters_ft FLOAT, 
receivedouters_ft FLOAT, 
packagetypeid_int INTEGER, 
expectedunitpriceperouter_ft FLOAT, 
lastreceiptdate_date DATE, 
isorderlinefinalized_bol BOOLEAN, 
right_lasteditedby_int INTEGER
'''
cs.execute(sql12)


In [123]:
sql13 = '''
UPDATE poh_raw SET 
purchaseorderid_int = CAST(purchaseorderid AS INTEGER), 
supplierid_int = CAST(supplierid AS INTEGER), 
orderdate_date = TRY_TO_DATE(orderdate), 
deliverymethodid_int = CAST(deliverymethodid AS INTEGER), 
contactpersonid_int = CAST(contactpersonid AS INTEGER), 
expecteddeliverydate_date = TRY_TO_DATE(expecteddeliverydate), 
isorderfinalized_bol = CAST(isorderfinalized AS BOOLEAN), 
lasteditedby_int = CAST(lasteditedby AS INTEGER), 
purchaseorderlineid_int = CAST(purchaseorderlineid AS INTEGER), 
stockitemid_int = CAST(stockitemid AS INTEGER), 
orderedouters_ft = CAST(orderedouters AS FLOAT), 
receivedouters_ft = CAST(receivedouters AS FLOAT), 
packagetypeid_int = CAST(packagetypeid AS INTEGER), 
expectedunitpriceperouter_ft = CAST(expectedunitpriceperouter AS FLOAT), 
lastreceiptdate_date = TRY_TO_DATE(lastreceiptdate), 
isorderlinefinalized_bol = CAST(isorderlinefinalized AS BOOLEAN), 
right_lasteditedby_int = CAST(right_lasteditedby AS INTEGER)
'''
cs.execute(sql13)

In [124]:
#delete original columns
sql14 = '''
ALTER TABLE poh_raw DROP COLUMN 
purchaseorderid, 
supplierid, 
orderdate, 
deliverymethodid, 
contactpersonid, 
expecteddeliverydate, 
isorderfinalized, 
lasteditedby, 
purchaseorderlineid, 
stockitemid, 
orderedouters, 
receivedouters, 
packagetypeid, 
expectedunitpriceperouter, 
lastreceiptdate, 
isorderlinefinalized, 
right_lasteditedby
'''
cs.execute(sql14)

In [125]:
#rename the new columns into original names
cs.execute("ALTER TABLE poh_raw RENAME COLUMN purchaseorderid_int TO purchaseorderid")
cs.execute("ALTER TABLE poh_raw RENAME COLUMN supplierid_int TO supplierid")
cs.execute("ALTER TABLE poh_raw RENAME COLUMN orderdate_date TO orderdate")
cs.execute("ALTER TABLE poh_raw RENAME COLUMN deliverymethodid_int TO deliverymethodid")
cs.execute("ALTER TABLE poh_raw RENAME COLUMN contactpersonid_int TO contactpersonid")
cs.execute("ALTER TABLE poh_raw RENAME COLUMN expecteddeliverydate_date TO expecteddeliverydate")
cs.execute("ALTER TABLE poh_raw RENAME COLUMN isorderfinalized_bol TO isorderfinalized")
cs.execute("ALTER TABLE poh_raw RENAME COLUMN lasteditedby_int TO lasteditedby")
cs.execute("ALTER TABLE poh_raw RENAME COLUMN purchaseorderlineid_int TO purchaseorderlineid")
cs.execute("ALTER TABLE poh_raw RENAME COLUMN stockitemid_int TO stockitemid")
cs.execute("ALTER TABLE poh_raw RENAME COLUMN orderedouters_ft TO orderedouters")
cs.execute("ALTER TABLE poh_raw RENAME COLUMN receivedouters_ft TO receivedouters")
cs.execute("ALTER TABLE poh_raw RENAME COLUMN packagetypeid_int TO packagetypeid")
cs.execute("ALTER TABLE poh_raw RENAME COLUMN expectedunitpriceperouter_ft TO expectedunitpriceperouter")
cs.execute("ALTER TABLE poh_raw RENAME COLUMN lastreceiptdate_date TO lastreceiptdate")
cs.execute("ALTER TABLE poh_raw RENAME COLUMN isorderlinefinalized_bol TO isorderlinefinalized")
cs.execute("ALTER TABLE poh_raw RENAME COLUMN right_lasteditedby_int TO right_lasteditedby")

In [ ]:
cs.execute("ALTER TABLE poh_raw RENAME TO purchaseorderheader_organized")

In [127]:
#Q2
sql21 = '''
CREATE TABLE purchaseorderheader AS 
SELECT *, SUM(receivedouters * expectedunitpriceperouter) OVER (PARTITION BY purchaseorderid) AS poamount 
FROM purchaseorderheader_organized
'''
cs.execute(sql21)

In [131]:
#Q3
cs.execute("CREATE STAGE st_internal_stage")

In [132]:
cs.execute("PUT file:///home/jovyan/sql_etl_project/Supplier_Transactions_XML.xml @st_internal_stage")

In [133]:
#create a staging table and load the xml data
cs.execute("CREATE OR REPLACE TABLE st_raw_staging (xml_data VARIANT)")
cs.execute("COPY INTO st_raw_staging FROM @st_internal_stage FILE_FORMAT = (TYPE = 'XML' STRIP_OUTER_ELEMENT = TRUE)")

In [54]:
cs.execute("SELECT xml_data FROM st_raw_staging")

In [141]:
#create table for final loading
sql31 = '''
CREATE OR REPLACE TABLE st_raw (
suppliertransactionid INTEGER, 
supplierid INTEGER, 
transactiontypeid INTEGER, 
purchaseorderid INTEGER, 
paymentmethodid INTEGER, 
supplierinvoicenumber INTEGER, 
transactiondate DATE, 
amountexcludingtax FLOAT, 
taxamount FLOAT, 
transactionamount FLOAT, 
outstandingbalance FLOAT, 
finalizationdate DATE, 
isfinalized BOOLEAN, 
lasteditedby INTEGER, 
lasteditedwhen VARCHAR
)
'''
cs.execute(sql31)

In [142]:
#insert into the new table
sql32 = '''
INSERT INTO st_raw (
suppliertransactionid, 
supplierid, 
transactiontypeid, 
purchaseorderid, 
paymentmethodid, 
supplierinvoicenumber, 
transactiondate, 
amountexcludingtax, 
taxamount, 
transactionamount, 
outstandingbalance, 
finalizationdate, 
isfinalized, 
lasteditedby, 
lasteditedwhen
) 
SELECT 
XMLGET(xml_data, 'SupplierTransactionID'):"$"::INTEGER AS suppliertransactionid, 
XMLGET(xml_data, 'SupplierID'):"$"::INTEGER AS supplierid, 
XMLGET(xml_data, 'TransactionTypeID'):"$"::INTEGER AS transactiontypeid, 
NULLIF(XMLGET(xml_data, 'PurchaseOrderID'):"$", '') AS purchaseorderid, 
XMLGET(xml_data, 'PaymentMethodID'):"$"::INTEGER AS paymentmethodid, 
NULLIF(XMLGET(xml_data, 'SupplierInvoiceNumber'):"$", '') AS supplierinvoicenumber, 
XMLGET(xml_data, 'TransactionDate'):"$"::DATE AS transactiondate, 
XMLGET(xml_data, 'AmountExcludingTax'):"$"::FLOAT AS amountexcludingtax, 
XMLGET(xml_data, 'TaxAmount'):"$"::FLOAT AS taxamount, 
XMLGET(xml_data, 'TransactionAmount'):"$"::FLOAT AS transactionamount, 
XMLGET(xml_data, 'OutstandingBalance'):"$"::FLOAT AS outstandingbalance, 
NULLIF(XMLGET(xml_data, 'FinalizationDate'):"$", '') AS finalizationdate, 
XMLGET(xml_data, 'IsFinalized'):"$"::STRING AS isfinalized, 
XMLGET(xml_data, 'LastEditedBy'):"$"::INTEGER AS lasteditedby, 
XMLGET(xml_data, 'LastEditedWhen'):"$"::STRING AS lasteditedwhen 
FROM st_raw_staging
'''
cs.execute(sql32)

In [ ]:
#test null value
cs.execute('''INSERT INTO st_raw (finalizationdate) SELECT XMLGET(xml_data, 'FinalizationDate'):"$"::DATE FROM st_raw_staging''') 

In [143]:
cs.execute("ALTER TABLE st_raw RENAME TO invoice")

In [11]:
#Q4 & Q5
sql41 = '''
CREATE VIEW poamount_specify AS
SELECT 
purchaseorderid, 
supplierid, 
SUM(receivedouters * expectedunitpriceperouter) as total
FROM purchaseorderheader 
GROUP BY purchaseorderid, supplierid
ORDER BY purchaseorderid, supplierid
'''
cs.execute(sql41)

In [13]:
sql51 = '''
CREATE VIEW purchase_orders_and_invoices AS 
SELECT 
ps.supplierid, 
ps.purchaseorderid,
amountexcludingtax,
total as poamount,
(amountexcludingtax - total) AS invoiced_vs_quoted, 
transactiondate 
FROM poamount_specify ps 
JOIN invoice i 
ON ps.supplierid = i.supplierid AND ps.purchaseorderid = i.purchaseorderid
ORDER BY ps.supplierid, ps.purchaseorderid
'''
cs.execute(sql51)

In [12]:
#需要改的
cs.execute("DROP VIEW purchase_orders_and_invoices")

In [6]:
#Q6
#download the dataset from pgsql database into the local
import psycopg2
import csv

connection = psycopg2.connect(
    host = '127.0.0.1',
    database = 'WestCoastImporters',
    user = 'jovyan',
    password = 'postgres',
    port = '8765'
)

cursor = connection.cursor()

cursor.execute("SELECT * FROM supplier_case")
rows = cursor.fetchall()

with open('/home/jovyan/sql_etl_project/supplier_case.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerow([i[0] for i in cursor.description])  
    writer.writerows(rows)

cursor.close()
connection.close()


In [55]:
cs.execute("CREATE STAGE sc_internal_stage")

In [60]:
sql61 = '''
CREATE OR REPLACE TABLE supplier (
supplierid INTEGER, 
suppliername VARCHAR, 
suppliercategoryid INTEGER, 
primarycontactpersonid INTEGER, 
alternatecontactpersonid INTEGER, 
deliverymethodid INTEGER, 
postalcityid STRING, 
supplierreference VARCHAR, 
bankaccountname VARCHAR, 
bankaccountbranch VARCHAR, 
bankaccountcode INTEGER, 
bankaccountnumber NUMERIC, 
bankinternationalcode INTEGER, 
paymentdays INTEGER, 
internalcomments VARCHAR, 
phonenumber VARCHAR, 
faxnumber VARCHAR, 
websiteurl VARCHAR, 
deliveryaddressline1 VARCHAR, 
deliveryaddressline2 VARCHAR, 
deliverypostalcode STRING, 
deliverylocation VARCHAR, 
postaladdressline1 VARCHAR, 
postaladdressline2 VARCHAR, 
postalpostalcode STRING, 
lasteditedby INTEGER, 
validfrom VARCHAR, 
validto VARCHAR
)
'''
cs.execute(dql61)

In [57]:
cs.execute("PUT file:///home/jovyan/sql_etl_project/supplier_case.csv @sc_internal_stage")

In [61]:
cs.execute('''COPY INTO supplier from @sc_internal_stage FILE_FORMAT = csv_format''')

In [52]:
#Q7
#load zcta txt file
cs.execute("CREATE STAGE zcta_internal_stage")

In [36]:
cs.execute("CREATE OR REPLACE TABLE zcta (geoid STRING, aland INTEGER, awater INTEGER, aland_sqmi FLOAT, awater_sqmi FLOAT, intptlat FLOAT, intptlong FLOAT)")

In [53]:
cs.execute("PUT file:///home/jovyan/sql_etl_project/2021_Gaz_zcta_national.txt @zcta_internal_stage")

In [37]:
cs.execute('''COPY INTO zcta from @zcta_internal_stage FILE_FORMAT = (TYPE = 'CSV' FIELD_DELIMITER = '\t' SKIP_HEADER = 1)''')

In [65]:
#extract the geolocation of each supplier 
sql71 = '''
CREATE VIEW supplier_location AS 
SELECT b.lat, b.lon, b.geoid 
FROM (SELECT DISTINCT postalpostalcode as postalcode FROM supplier) a 
JOIN (SELECT intptlat as lat, intptlong as lon, geoid FROM zcta) b 
ON a.postalcode = b.geoid")
'''
cs.execute(sql71)

In [28]:
#find the weather station that is closest to suppliers' geolocation
sql72 = '''
CREATE VIEW find_closest_station AS
SELECT b.geoid, a.noaa_weather_station_id, st_distance(st_makepoint(a.longitude, a.latitude), st_makepoint(b.lon, b.lat)) AS distance 
FROM WEATHER__ENVIRONMENT.CYBERSYN.noaa_weather_station_index a 
JOIN supplier_location b 
ON st_distance(st_makepoint(a.longitude, a.latitude), st_makepoint(b.lon, b.lat)) < 50000
'''
cs.execute(sql72)

In [23]:
#get the highest temperatures by date measured by the weather stations which have the closest distance to the suppliers' geolocation
sql73 = '''
CREATE VIEW temp_overview AS
SELECT wmt.noaa_weather_station_id, variable_name, date, value AS temperature, unit
FROM weather__environment.cybersyn.noaa_weather_metrics_timeseries wmt
JOIN
(SELECT geoid, noaa_weather_station_id
FROM find_closest_station
WHERE distance IN (
SELECT shortest_distance FROM (
SELECT geoid, MIN(distance) AS shortest_distance
FROM find_closest_station
GROUP BY geoid) a)) b
ON wmt.noaa_weather_station_id = b.noaa_weather_station_id
WHERE variable_name = 'Maximum Temperature'
'''
cs.execute(sql73)

In [52]:
#create view of the suppliers' geolocation and the corresponding weather station ids to be better prepared for join
sql74 = '''
CREATE VIEW shortest_distance_station AS
SELECT geoid, noaa_weather_station_id FROM find_closest_station
WHERE noaa_weather_station_id IN (
SELECT noaa_weather_station_id FROM find_closest_station WHERE distance IN (SELECT shortest_distance FROM
(SELECT geoid, MIN(distance) AS shortest_distance FROM find_closest_station GROUP BY geoid) a))
'''
cs.execute(sql74)

In [26]:
#create view supplier_zip_code_weather
sql75 = '''
CREATE VIEW supplier_zip_code_weather AS
SELECT sds.geoid AS zip_code, tem.date, tem.temperature, tem.unit FROM shortest_distance_station sds 
JOIN temp_overview tem ON sds.noaa_weather_station_id = tem.noaa_weather_station_id
ORDER BY date
'''
cs.execute(sql75)

In [28]:
#Q8
sql81 = '''
SELECT supplierid, suppliername
FROM supplier s
JOIN 
(SELECT zip_code
FROM supplier_zip_code_weather szw
JOIN purchase_orders_and_invoices poi
ON szw.date = poi.transactiondate) a
ON s.postalpostalcode = a.zip_code
'''
cs.execute(sql81)